In [2]:
import pandas as pd
import pystan
import numpy as np

In [3]:
dataframe = pd.read_csv('linear-regression-model/linear-data.csv')

In [4]:
dataframe.head()

,template_id,coadd_objects_id,r11,r22,template_type,median_z,mean_z,minchi2,mode_z,z_sigma68,z_sigma,kl_post_prior,z_mc
0,46,3009461516,0.679885,-0.354576,5.000000,2.192059,2.215981,47.672108,2.07,0.170656,0.273618,3.111425,2.412117
1,20,3009461517,-0.896715,-0.373454,2.222222,0.639860,0.638648,0.362409,0.66,0.050641,0.064265,1.501075,0.697164
2,0,3009461518,0.790496,0.756900,0.000000,0.165350,0.195981,14.404441,0.17,0.016207,0.089402,2.385706,0.196782
3,46,3009461519,1.881596,-3.509992,5.000000,0.573707,0.698522,0.006209,0.61,0.383936,0.604144,0.582606,2.784782
4,45,3009461520,13.398794,-8.360839,4.900000,0.731808,0.900801,0.001354,0.59,0.560707,0.621029,0.078241,0.923708


In [5]:
df = dataframe[['template_id', 'median_z', 'minchi2', 'z_sigma']]
df.head()

,template_id,median_z,minchi2,z_sigma
0,46,2.192059,47.672108,0.273618
1,20,0.639860,0.362409,0.064265
2,0,0.165350,14.404441,0.089402
3,46,0.573707,0.006209,0.604144
4,45,0.731808,0.001354,0.621029


In [6]:
N = df.shape[0]

log_minchi2 = np.log(df["minchi2"])
z_sigma = df["z_sigma"]
median_z = df["median_z"]

In [7]:
data = {'N': N, 
        'log_minchi2': log_minchi2,
        'median_z': median_z,
        'z_sigma': z_sigma,}

In [16]:
import pystan

stan_code = '''
data {
    int<lower=0> N;
    vector[N] log_minchi2;
    vector[N] median_z;
    vector[N] z_sigma;
}
parameters {
    real beta_0;
    real beta_1;
    real beta_2;
    real<lower=0> sigma;
}
model {
    beta_0 ~ normal(0, 10);
    beta_1 ~ normal(0, 10);
    beta_2 ~ normal(0, 10);
    sigma ~ normal(0, 5);

    for (i in 1:N)
        median_z[i] ~ normal(beta_0 + beta_1 * log_minchi2[i] + beta_2 * z_sigma[i], sigma);
}
'''

# Compile the model
sm = pystan.StanModel(model_code=stan_code)

# Fit the model
fit = sm.sampling(data=data, iter=1000, chains=4)

# Print the fit
print(fit)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_663c6f6211079cb21bf81dd2e34f5df0 NOW.


DistutilsPlatformError: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/